In [2]:
import torch
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import torch.nn.functional as F
import math

In [ ]:
model_utils = {
    "resenet50": "https://download.pytorch.org/models/resnet50-19c8e357.pth"
}

def conv3x3(in_planes, out_planes, stride=1):
    """3*3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

class Bottleneck(nn.Module):
    # Resnet-B
    expansion = 4
    
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.
        

# 检测分支

#### nn.parameter

可以把这个函数理解为类型转换函数，将一个不可训练的数据类型Tensor转换成可以训练的数据类型parameter,并将这个parameter绑定到这个module里面；

In [17]:
class ScaleExp(nn.Module):
    """
    定义一个指数缩放模块
    """
    def __init__(self, init_value=1.0):
        super(ScaleExp, self).__init__()
        self.scale = nn.Parameter(torch.tensor([init_value], dtype=torch.float32))  # nn.parameter 将一个不可训练的tensor转换乘可以训练的类型parameter,并将这个parameter绑定到这个module里面； 
    
    def forward(self, x):
        return torch.exp(x * self.scale)  # 乘一个可以训练的缩放因子 scale

In [18]:
class ClsCntRegHead(nn.Module):
    """检测分支

    Args:
        nn (_type_): _description_
    """
    def __init__(self, in_channel, class_num, GN=True, cnt_on_reg=True, prior=0.01):
        super(ClsCntRegHead, self).__init__()
        self.prior = prior
        self.class_num = class_num
        self.cnt_on_reg = cnt_on_reg
        
        # 1 ========================从fpn到head的侧连===========================
        cls_branch = []
        reg_branch = []
        for i in range(4):
            # cls_branch: conv--gn--relu  这里的卷积都不改变图像尺寸
            cls_branch.append(nn.Conv2d(in_channel, in_channel, kernel_size=3, padding=1, bias = True))
            if GN:
                reg_branch.append(nn.GroupNorm(32, in_channel))
            cls_branch.append(nn.ReLU(True))
            
            # reg_branch: conv--gn--relu  这里的卷积都不改变图像尺寸
            reg_branch.append(nn.Conv2d(in_channel, in_channel, kernel_size=3, padding=1, bias = True))
            if GN:
                reg_branch.append(nn.GroupNorm(32, in_channel))
            reg_branch.append(nn.ReLU(True))
        
        # 1.1 分类网络分支
        self.cls_conv = nn.Sequential(*cls_branch)  # 将cls_branch中的参数，一个个拆解成独立的参数；列表或者元组之前加一个*，字典前加两个**
        # 1.2 回归网络分支
        self.reg_conv = nn.Sequential(*reg_branch)
        print(' Bypasses of the detection head;')
        print(self.cls_conv, '\n')
        print(self.reg_conv)
        # =======================================================================
        
        # 2 ========================网络输出=====================================
        # 2.1 网络分类路径输出
        self.cls_logits = nn.Conv2d(in_channel, class_num, kernel_size=3, padding=1)
        # 2.2 网络回归路径输出
        self.reg_pred = nn.Conv2d(in_channel, 4, kernel_size=3, padding=1)
        # 2.3 目标中心输出
        self.cnt_logits = nn.Conv2d(in_channel, 1, kernel_size=3, padding=1)
        # =======================================================================
        
        # 3. 网络参数初始化
        self.apply(self.init_conv_RandomNormal)
        nn.init.constant_(self.cls_logits.bias, - math.log((1-prior)/prior))
        
        # 4. 实例化五个缩放层
        self.scale_exp = nn.ModuleList([ScaleExp(1.0) for _ in range(5)])        
        
    def init_conv_RandomNormal(self, module, std=0.01):
        if isinstance(module, nn.Conv2d):
            nn.init.normal_(module.weight, std=std)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)
                
    def forward(self, inputs):
        """_summary_

        Args:
            inputs ([[p3~p7]): _description_
        """
        cls_logits = []
        cnt_logits = []
        reg_preds = []
        for index, P in enumerate(inputs):
            # 所有的P层都经过cls_conv(),所以这里，模块cls_conv()的参数是共享的。
            cls_conv_out = self.cls_conv(P)
            # 所有的p层都经过reg_conv(),所以这里，模块reg_conv()的参数是共享的。
            reg_conv_out = self.reg_conv(P)
            
            cls_logits.append(self.cls_logits(cls_conv_out))
            
            if not self.cnt_on_reg:  # 中心回归放在哪一个分支上，是cls_conv_out, 还是reg_conv_out分支
                cnt_logits.append(self.cnt_logits(cls_conv_out))
            else:
                cnt_logits.append(self.cnt_logits(reg_conv_out))  # 中心回归默认放在reg_conv_out分支
            
            reg_preds.append(self.scale_exp[index](self.reg_pred(reg_conv_out)))
        return cls_logits, cnt_logits, reg_preds  # 每一个返回的list都有5个分量，对应P3~P7的卷积输出；
        

In [19]:
head = ClsCntRegHead(256, 20)

 Bypasses of the detection head;
Sequential(
  (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): ReLU(inplace=True)
  (6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
) 

Sequential(
  (0): GroupNorm(32, 256, eps=1e-05, affine=True)
  (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): GroupNorm(32, 256, eps=1e-05, affine=True)
  (3): ReLU(inplace=True)
  (4): GroupNorm(32, 256, eps=1e-05, affine=True)
  (5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): GroupNorm(32, 256, eps=1e-05, affine=True)
  (7): ReLU(inplace=True)
  (8): GroupNorm(32, 256, eps=1e-05, affine=True)
  (9): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (10): 